In [1]:
import sys
sys.path.append('../../software/main_distortion_generation/')
from gen import *
import vis_utils as vu
import PIL
import meshplot as mp
import pandas as pd
from matplotlib import cm

In [2]:
def split_tris_snapped(v,f,fid, bc):
    pins = (v[f[fid]]*bc[:,:,None]).sum(axis=1)
    f1, cnt = [], 0
    nv = len(v)
    f = f.copy()
    for n, (ti, b) in enumerate(zip(fid,pins)):
        a0,a1,a2 = f[ti]
        f[ti] = [n + nv, a0, a1]
        f1.append([n + nv, a1, a2])
        f1.append([n + nv, a2, a0])
    v_c, f_c =  np.concatenate([v, pins]), np.concatenate([f, f1])
    v1,inI, inJ, f1 = igl.remove_duplicate_vertices(v_c, f_c, 1e-2)
    #print(inI.shape, inJ.shape)
    return v1,f1, inJ[(np.arange(len(v), len(v) +len(pins)))]


In [3]:
def split_tris(v,f,fid, bc):
    pins = (v[f[fid]]*bc[:,:,None]).sum(axis=1)
    f1, cnt = [], 0
    nv = len(v)
    f = f.copy()
    for n, (ti, b) in enumerate(zip(fid,pins)):
        a0,a1,a2 = f[ti]
        f[ti] = [n + nv, a0, a1]
        f1.append([n + nv, a1, a2])
        f1.append([n + nv, a2, a0])
    return np.concatenate([v, pins]), np.concatenate([f, f1]), np.arange(len(v), len(v) + len(pins), dtype=int)

def region_average(jod_dict):
    for i in range(6,19):
        avg = (jod_dict[i] + jod_dict[i+13])/2
        jod_dict[i] = avg
        jod_dict[i+13] = avg

def colorbar(mn, ma, ticks = None):
    ra = ma - mn
    import matplotlib.pyplot as plt
    import numpy as np
    fig,ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(np.array([[0,1]])*ra + mn, aspect=1e-4)
    ax.axis('off')
    cbar = plt.colorbar(im,orientation='horizontal', location='top')
    #cbar.set_ticks(np.linspace(1,3,5))
    #cbar.ax.set_yticklabels(np.linspace(1,3,5))
    plt.show()
# colorbar(1,3)

In [4]:
def array_of_jod(jods, ind):
    l = np.zeros(len(ind))
    l[ind] = jods
    return l, np.sort(ind)

def interpolate(spl_v, spl_f, keys, vals):
    back_verts = np.where(spl_v[:,2]<-100)[0].astype(int)
    back_vals = 0*np.ones(len(back_verts))

    interp_vals = igl.harmonic_weights(spl_v, spl_f, 
                                       b = np.concatenate([keys, back_verts]),
                                       bc = np.concatenate([vals, back_vals]), 
                                       k = 2)
    return interp_vals


In [5]:
def read_jod_data(file, headx):
    table = pd.read_csv(file)
    if 'scene' in table:
        table = table[table['scene'] == headx]
    ref_id = (table[table.condition.str.contains('ref')].index[0])
    table['jod'] = table['jod'] - table.iloc[ref_id]['jod']
    # print(len(table))
    # assert len(table) == 191
    return table


In [ ]:
CSV_ROOT = '../../data/main_study/processed/bootstrap100/'

head_path = '../../software/main_distortion_generation'
headx = 'head1'
v,tv,_, f,tf,_ = igl.read_obj(f'{head_path}/data/{headx}.obj')
pts = np.load(f'{head_path}/data/landmarks.npy')
fid, bc = du.lift_uv(tv,tf,pts)
colors = np.asarray(PIL.ImageOps.flip(PIL.Image.open(f'{head_path}/data/{headx}.png')), dtype=np.float32)[:,:,:3]/255
spl_v, spl_f,vert_selected = split_tris(v,f,fid,bc)
table = read_jod_data(CSV_ROOT + f'head-final-perartifact-jod.csv',
             headx)

headx = 'head1'
v,tv,_, f,tf,_ = igl.read_obj('C:/Users/jzs/wrapref-in-artist-body-sown.obj')
pts = np.load('snapper-landmark.npy')
fid, bc = du.lift_uv(tv,tf,pts)
colors = np.asarray(PIL.ImageOps.flip(PIL.Image.open(f'data/{headx}.png')), dtype=np.float32)[:,:,:3]/255
spl_v, spl_f, vert_selected = split_tris_snapped(v,f,fid,bc)
#spl_v, spl_f = split_tris(v,f,fid,bc)

# Total Map

In [11]:
def extract_and_interpolate_values(subtable):
    def func(x):
        if '_' in x:
            return int(x.split('_')[-1][1:])
        else:
            return int(x[1:])
    jod_vals,jod_keys = array_of_jod(np.asarray(subtable.jod),
             list(map(func,
                     np.asarray(subtable['condition']))))
    if len(jod_keys) == 19:
        print('Note: auto complete left side of the face')
        jod_vals,jod_keys = np.concatenate([jod_vals, jod_vals[6:]]), np.concatenate([jod_keys, np.arange(6,6+13)+13])
    interp_vals = interpolate(spl_v, spl_f, keys = vert_selected,
                              vals = - np.clip(jod_vals,-5,0))
    print(interp_vals.max(), interp_vals.min())
    return interp_vals

In [12]:
# Color Plot
arti_types = ['simp', 'smooth', 'noise', 'jpg','resample']

map_dicts = {}
for art in arti_types:
    st = table[table.condition.str.startswith(art)]
    map_dicts[art] = extract_and_interpolate_values(st)
joint = read_jod_data(CSV_ROOT + f'head-final-joint-jod.csv',
             headx)
map_dicts['total']= extract_and_interpolate_values(joint[joint.condition != 'ref'])

Note: auto complete left side of the face
1.3557097434937104 -0.15828210403932577
Note: auto complete left side of the face
1.2039471484570254 -0.15073909803515848
Note: auto complete left side of the face
1.2394597149855382 -0.10178958789345563
Note: auto complete left side of the face
1.186614325363427 -0.0005324806253699533
Note: auto complete left side of the face
2.5265648649322827 -0.0010059315128891537
Note: auto complete left side of the face
0.8559621288862863 -0.00023926523774258694


np.savez('map_dict.npz', **map_dicts)

In [ ]:
bc = igl.barycenter(spl_v,spl_f)

sliced_f = spl_f[bc[:,1] + bc[:,2] >-140]

spl_vn = igl.per_vertex_normals(spl_v,spl_f)

for k in arti_types:
    vw = mp.plot(spl_v,sliced_f, c = cm.viridis(map_dicts[k] / 1.5)[:,:3],
            shading=dict(wireframe=False, flat=False))
